<a href="https://colab.research.google.com/github/mbtejeswar/MetreSense/blob/dev/MetreSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install numpy pandas tensorflow keras matplotlib opencv-python
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


images_path = '/content/drive/MyDrive/Meter_Readings'
csv_path = '/content/drive/MyDrive/Meter_Readings.csv'


data = pd.read_csv(csv_path)


# Preprocess the images and labels
def load_images_and_labels(data, images_path):
    images = []
    labels = []
    for _, row in data.iterrows():
        img_path = os.path.join(images_path, row['image_name'])
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if img is not None:
            img = cv2.resize(img, (128, 64))  # Resize to 128x64
            images.append(img)
            labels.append([int(d) for d in str(row['label'])])  # Convert label to digits
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_images_and_labels(data, images_path)

# Normalize image pixel values
images = images / 255.0  # Normalize to [0, 1]
images = images.reshape(-1, 64, 128, 1)  # Add channel dimension for grayscale

# Prepare labels for multi-digit output
num_digits = labels.shape[1]  # Number of digits in the label
num_classes = 10  # Digits 0-9

labels = np.array([to_categorical(digit, num_classes) for label in labels for digit in label])
labels = labels.reshape(-1, num_digits, num_classes)  # Reshape for multi-digit output

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the CNN model
def create_model(num_digits, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 128, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(num_digits * num_classes, activation='softmax'))  # Output layer

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create and compile the model
model = create_model(num_digits=num_digits, num_classes=num_classes)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_val, y_val))

# Save the model
model.save('/content/drive/MyDrive/meter_reading_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Meter_Readings.csv'

In [6]:
!ls /content/drive/MyDrive/Meter_Readings

IMG_20210201_174953.jpg  IMG_20210201_175641.jpg  IMG_20210201_175808.jpg  IMG_20210201_175944.jpg
IMG_20210201_174956.jpg  IMG_20210201_175659.jpg  IMG_20210201_175812.jpg  IMG_20210201_175946.jpg
IMG_20210201_174959.jpg  IMG_20210201_175701.jpg  IMG_20210201_175814.jpg  IMG_20210201_175948.jpg
IMG_20210201_175001.jpg  IMG_20210201_175704.jpg  IMG_20210201_175836.jpg  IMG_20210201_175951.jpg
IMG_20210201_175004.jpg  IMG_20210201_175707.jpg  IMG_20210201_175838.jpg  IMG_20210201_175954.jpg
IMG_20210201_175008.jpg  IMG_20210201_175709.jpg  IMG_20210201_175841.jpg  IMG_20210201_175956.jpg
IMG_20210201_175050.jpg  IMG_20210201_175728.jpg  IMG_20210201_175844.jpg  IMG_20210201_180016.jpg
IMG_20210201_175053.jpg  IMG_20210201_175731.jpg  IMG_20210201_175847.jpg  IMG_20210201_180019.jpg
IMG_20210201_175056.jpg  IMG_20210201_175734.jpg  IMG_20210201_175908.jpg  IMG_20210201_180021.jpg
IMG_20210201_175059.jpg  IMG_20210201_175737.jpg  IMG_20210201_175911.jpg  IMG_20210201_180023.jpg
IMG_202102